In [1]:
import pandas as pd

# Load the parquet file into a DataFrame
df = pd.read_parquet('../test')
df

,user_id,merchant_abn,dollar_value,order_id,order_datetime
0,1,28000487688,133.226894,0c37b3f7-c7f1-48cb-bcc7-0a58e76608ea,2021-02-28
1,18485,62191208634,79.131400,9e18b913-0465-4fd4-92fd-66d15e65d93c,2021-02-28
2,1,83690644458,30.441348,40a2ff69-ea34-4657-8429-df7ca957d6a1,2021-02-28
3,18488,39649557865,962.813341,f4c1a5ae-5b76-40d0-ae0f-cb9730ac325a,2021-02-28
4,2,80779820715,48.123977,cd09bdd6-f56d-489f-81ea-440f4bda933c,2021-02-28
...,...,...,...,...,...
14195500,11136,77698107389,400.792069,5e68cb94-3eab-4342-83fa-65263eff26b8,2022-10-26
14195501,11136,24852446429,5.384811,ba2ce936-ae49-4b70-a4b1-e8f8bacaefb3,2022-10-26
14195502,11136,63685007785,48.129887,a6194c61-ae7a-4037-a425-2dd7cda42fb7,2022-10-26
14195503,11137,85502310765,4.768752,72a3639e-25d5-4bcc-9e8b-26e737b8c0e4,2022-10-26


In [2]:
df = df.drop(columns=['user_id', 'order_id'])
df.dtypes

merchant_abn         int64
dollar_value       float64
order_datetime    category
dtype: object

In [3]:
# Convert order_datetime to string, then to datetime
df['order_datetime'] = pd.to_datetime(df['order_datetime'].astype(str))
df.dtypes

merchant_abn               int64
dollar_value             float64
order_datetime    datetime64[ns]
dtype: object

In [4]:
# Set order_datetime as the index
df.set_index('order_datetime', inplace=True)

# Aggregate dollar_value for each merchant for each month
weekly_aggregation = df.groupby('merchant_abn').resample('W').sum()

weekly_aggregation.drop(columns=['merchant_abn'], inplace=True)

In [5]:
weekly_aggregation


dollar_value
merchant_abn order_datetime              
10023283211  2021-02-28        701.566638
             2021-03-07       6362.419840
             2021-03-14       4706.012141
             2021-03-21       3536.843553
             2021-03-28       6796.619996
...                                   ...
99990536339  2022-09-18        586.143138
             2022-09-25          0.000000
             2022-10-02          0.000000
             2022-10-09          0.000000
             2022-10-16        578.396323

[373738 rows x 1 columns]

In [6]:
# Create a list of DataFrames, each corresponding to a unique merchant_abn
merchant_dfs = [{merchant_abn: weekly_aggregation.loc[merchant_abn].reset_index()} for merchant_abn in weekly_aggregation.index.get_level_values('merchant_abn').unique()]
merchant_dfs

[{10023283211:    order_datetime  dollar_value
  0      2021-02-28    701.566638
  1      2021-03-07   6362.419840
  2      2021-03-14   4706.012141
  3      2021-03-21   3536.843553
  4      2021-03-28   6796.619996
  ..            ...           ...
  83     2022-10-02  10731.916071
  84     2022-10-09   9379.854136
  85     2022-10-16  10905.389070
  86     2022-10-23   7739.163021
  87     2022-10-30   6434.632797
  
  [88 rows x 2 columns]},
 {10142254217:    order_datetime  dollar_value
  0      2021-02-28     10.886926
  1      2021-03-07   1138.039207
  2      2021-03-14    616.629903
  3      2021-03-21   1132.443977
  4      2021-03-28   1491.259886
  ..            ...           ...
  83     2022-10-02   1281.167336
  84     2022-10-09   2423.409637
  85     2022-10-16   1949.294210
  86     2022-10-23   2548.415792
  87     2022-10-30    719.994180
  
  [88 rows x 2 columns]},
 {10165489824:    order_datetime  dollar_value
  0      2021-12-05  20636.897020
  1      2021-12-12

In [8]:
'''import matplotlib.pyplot as plt

# Plot the first 5 merchants' weekly revenue on the same plot
plt.figure(figsize=(15, 7))
for i in range(10):
    merchant_abn, df = list(merchant_dfs[i].items())[0]
    plt.plot(df['ds'], df['y'], label=f'Merchant {merchant_abn}')

plt.xlabel('Date')
plt.ylabel('Weekly Revenue')
plt.title('Weekly Revenue for First 5 Merchants')
plt.legend()
plt.show()''''''

KeyError: 'ds'

<Figure size 1500x700 with 0 Axes>

In [9]:
from prophet import Prophet

In [10]:
dfone = merchant_dfs[0][10023283211]
dfone

,order_datetime,dollar_value
0,2021-02-28,701.566638
1,2021-03-07,6362.419840
2,2021-03-14,4706.012141
3,2021-03-21,3536.843553
4,2021-03-28,6796.619996
...,...,...
83,2022-10-02,10731.916071
84,2022-10-09,9379.854136
85,2022-10-16,10905.389070
86,2022-10-23,7739.163021


In [11]:
dfone = merchant_dfs[0][10023283211]
dfone.rename(columns={'order_datetime': 'ds', 'dollar_value': 'y'}, inplace=True)
model = Prophet()
model.fit(dfone)

23:11:15 - cmdstanpy - INFO - Chain [1] start processing
23:11:15 - cmdstanpy - INFO - Chain [1] done processing


In [12]:
# Initialize a dictionary to store the Prophet models and CWGR for each merchant
merchant_models = {}
merchant_cwgr = {}

# Iterate through each merchant's data in merchant_dfs
for merchant_data in merchant_dfs:
    for merchant_abn, df in merchant_data.items():
        # Prepare the data for Prophet
        df.rename(columns={'order_datetime': 'ds', 'dollar_value': 'y'}, inplace=True)
        
        # Check if the DataFrame has less than or equal to 1 entry
        if len(df) <= 1:
            print(f"Skipping Merchant {merchant_abn} due to insufficient data.")
            merchant_cwgr[merchant_abn] = 0
            continue

        # Initialize and train the Prophet model
        model = Prophet()
        print(f"Training model for Merchant {merchant_abn}...")
        model.fit(df)
        
        # Store the trained model
        merchant_models[merchant_abn] = model
        
        # Make future dataframe for prediction
        future = model.make_future_dataframe(periods=13, freq='W')
        forecast = model.predict(future)
        
        # Calculate CWGR
        initial_value = df['y'].iloc[0]
        final_value = forecast['yhat'].iloc[-1]
        weeks = len(forecast)
        cwgr = ((final_value / initial_value) ** (1 / weeks)) - 1
        
        # Store the CWGR
        merchant_cwgr[merchant_abn] = cwgr

merchant_cwgr

23:11:16 - cmdstanpy - INFO - Chain [1] start processing
23:11:16 - cmdstanpy - INFO - Chain [1] done processing
23:11:16 - cmdstanpy - INFO - Chain [1] start processing


Training model for Merchant 10023283211...
Training model for Merchant 10142254217...


23:11:16 - cmdstanpy - INFO - Chain [1] done processing
23:11:16 - cmdstanpy - INFO - Chain [1] start processing
23:11:16 - cmdstanpy - INFO - Chain [1] done processing
C:\Users\xXDra\AppData\Local\Temp\ipykernel_41772\126397658.py:33: RuntimeWarning: invalid value encountered in scalar power
  cwgr = ((final_value / initial_value) ** (1 / weeks)) - 1
23:11:16 - cmdstanpy - INFO - Chain [1] start processing
23:11:16 - cmdstanpy - INFO - Chain [1] done processing


Training model for Merchant 10165489824...
Training model for Merchant 10187291046...


23:11:16 - cmdstanpy - INFO - Chain [1] start processing
23:11:16 - cmdstanpy - INFO - Chain [1] done processing
23:11:16 - cmdstanpy - INFO - Chain [1] start processing


...

Training model for Merchant 99974311662...
Training model for Merchant 99976658299...


23:21:52 - cmdstanpy - INFO - Chain [1] start processing
23:21:52 - cmdstanpy - INFO - Chain [1] done processing
23:21:52 - cmdstanpy - INFO - Chain [1] start processing
23:21:52 - cmdstanpy - INFO - Chain [1] done processing


Training model for Merchant 99987905597...
Skipping Merchant 99989036621 due to insufficient data.
Training model for Merchant 99990536339...


C:\Users\xXDra\AppData\Local\Temp\ipykernel_41772\126397658.py:33: RuntimeWarning: invalid value encountered in scalar power
  cwgr = ((final_value / initial_value) ** (1 / weeks)) - 1


{10023283211: 0.026742957147423585,
 10142254217: 0.05112403605823812,
 10165489824: nan,
 10187291046: 0.00016726579825410148,
 10192359162: 0.007905872287638394,
 10206519221: 0.0325751806952832,
 10255988167: -0.0006816474311490683,
 10264435225: 0.02062172739039836,
 10279061213: 0.010483465938006864,
 10323485998: 0.0247155562942718,
 10342410215: 0.0209621946555143,
 10346855916: -0.024431905881255878,
 10364012396: 0.006369498274805707,
 10385011947: -0.001593618637924954,
 10385163239: -0.0019530980409676602,
 10385250025: 0.018135151457331578,
 10404542215: 0,
 10430380319: 0.0017245680007542674,
 10441711491: -0.04127459470241235,
 10462560289: 0.039985715114438936,
 10463252268: -0.005391238450552671,
 10487253336: 0.024147338113975403,
 10530696903: 0.012431046434745019,
 10545955006: 0.0010396125994671657,
 10553813474: 0.009817706707567764,
 10596295795: -0.0029369416805344883,
 10618089367: 0.04040797366262905,
 10648956813: 0.024914289104042364,
 10651113986: 0.01080115

In [13]:
# Impute NaN CWGR values with 0
for merchant_abn, cwgr_value in merchant_cwgr.items():
    if pd.isna(cwgr_value):
        merchant_cwgr[merchant_abn] = 0

merchant_cwgr

{10023283211: 0.026742957147423585,
 10142254217: 0.05112403605823812,
 10165489824: 0,
 10187291046: 0.00016726579825410148,
 10192359162: 0.007905872287638394,
 10206519221: 0.0325751806952832,
 10255988167: -0.0006816474311490683,
 10264435225: 0.02062172739039836,
 10279061213: 0.010483465938006864,
 10323485998: 0.0247155562942718,
 10342410215: 0.0209621946555143,
 10346855916: -0.024431905881255878,
 10364012396: 0.006369498274805707,
 10385011947: -0.001593618637924954,
 10385163239: -0.0019530980409676602,
 10385250025: 0.018135151457331578,
 10404542215: 0,
 10430380319: 0.0017245680007542674,
 10441711491: -0.04127459470241235,
 10462560289: 0.039985715114438936,
 10463252268: -0.005391238450552671,
 10487253336: 0.024147338113975403,
 10530696903: 0.012431046434745019,
 10545955006: 0.0010396125994671657,
 10553813474: 0.009817706707567764,
 10596295795: -0.0029369416805344883,
 10618089367: 0.04040797366262905,
 10648956813: 0.024914289104042364,
 10651113986: 0.0108011505

In [14]:
# Sort the merchants by highest CWGR to lowest
sorted_merchant_cwgr = {k: v for k, v in sorted(merchant_cwgr.items(), key=lambda item: item[1], reverse=True)}
sorted_merchant_cwgr

{72519488277: 0.34429356468295236,
 20890410130: 0.08271043186505089,
 45888399278: 0.07418512535693855,
 50885695263: 0.07090364447777309,
 73546158413: 0.07011673811455399,
 48549026640: 0.06786862155354889,
 38710494876: 0.06622054037424863,
 95560742626: 0.06092284189904418,
 23055736061: 0.060837602578440064,
 34095207240: 0.05852364723969239,
 55123279982: 0.058449749138903995,
 60070647768: 0.05817586430645827,
 88795682549: 0.057778463356216614,
 62285108846: 0.057710524142983166,
 92308095254: 0.05768949620215835,
 37621714049: 0.05547709061739403,
 13226823813: 0.05530127272234031,
 17124978176: 0.054696509493319834,
 27639240514: 0.05337057424839475,
 71910092040: 0.05331289407307227,
 79269463423: 0.053257083259517835,
 41237771126: 0.052788008798113184,
 23399930700: 0.052431724482431497,
 57391651605: 0.0523960757992914,
 65202518558: 0.052226940196347105,
 11470993597: 0.05212995744709614,
 11137507330: 0.05197978520028479,
 10142254217: 0.05112403605823812,
 41747634618

In [15]:
# Convert the sorted dictionary to a DataFrame
sorted_merchant_cwgr_df = pd.DataFrame(list(sorted_merchant_cwgr.items()), columns=['merchant_abn', 'cwgr'])
sorted_merchant_cwgr_df

,merchant_abn,cwgr
0,72519488277,0.344294
1,20890410130,0.082710
2,45888399278,0.074185
3,50885695263,0.070904
4,73546158413,0.070117
...,...,...
4417,15293888435,-0.073221
4418,13467303030,-0.076551
4419,86201937910,-0.076596
4420,34205660775,-0.107480


In [16]:
# Count the number of weeks of data for each merchant
merchant_week_counts = weekly_aggregation.groupby('merchant_abn').size().reset_index(name='week_count')

# Merge the week counts back into the original DataFrame
weekly_aggregation = weekly_aggregation.reset_index().merge(merchant_week_counts, on='merchant_abn').set_index(['merchant_abn', 'order_datetime'])

weekly_aggregation

dollar_value  week_count
merchant_abn order_datetime                          
10023283211  2021-02-28        701.566638          88
             2021-03-07       6362.419840          88
             2021-03-14       4706.012141          88
             2021-03-21       3536.843553          88
             2021-03-28       6796.619996          88
...                                   ...         ...
99990536339  2022-09-18        586.143138          76
             2022-09-25          0.000000          76
             2022-10-02          0.000000          76
             2022-10-09          0.000000          76
             2022-10-16        578.396323          76

[373738 rows x 2 columns]

In [17]:
# Reset the index of weekly_aggregation to include merchant_abn as a column
weekly_aggregation_reset = weekly_aggregation.reset_index()

# Drop duplicate rows to ensure we only have one row per merchant_abn with the week_count
weekly_aggregation_unique = weekly_aggregation_reset[['merchant_abn', 'week_count']].drop_duplicates()

# Merge the DataFrames on merchant_abn
merged_df = pd.merge(sorted_merchant_cwgr_df, weekly_aggregation_unique, on='merchant_abn', how='left')

merged_df

,merchant_abn,cwgr,week_count
0,72519488277,0.344294,2
1,20890410130,0.082710,83
2,45888399278,0.074185,88
3,50885695263,0.070904,88
4,73546158413,0.070117,88
...,...,...,...
4417,15293888435,-0.073221,52
4418,13467303030,-0.076551,75
4419,86201937910,-0.076596,43
4420,34205660775,-0.107480,49


In [19]:
# Update the cwgr to 0 for merchants with a week_count of under 10
merged_df.loc[merged_df['week_count'] < 10, 'cwgr'] = 0

merged_df

,merchant_abn,cwgr,week_count
0,72519488277,0.000000,2
1,20890410130,0.082710,83
2,45888399278,0.074185,88
3,50885695263,0.070904,88
4,73546158413,0.070117,88
...,...,...,...
4417,15293888435,-0.073221,52
4418,13467303030,-0.076551,75
4419,86201937910,-0.076596,43
4420,34205660775,-0.107480,49


In [20]:
# Sort the merged_df DataFrame by the 'cwgr' column in descending order
sorted_merged_df = merged_df.sort_values(by='cwgr', ascending=False)
sorted_merged_df

,merchant_abn,cwgr,week_count
1,20890410130,0.082710,83
2,45888399278,0.074185,88
3,50885695263,0.070904,88
4,73546158413,0.070117,88
5,48549026640,0.067869,23
...,...,...,...
4417,15293888435,-0.073221,52
4418,13467303030,-0.076551,75
4419,86201937910,-0.076596,43
4420,34205660775,-0.107480,49


While a high CwGR is desirable, make sure the growth is sustainable and not a result of temporary factors like seasonal sales or one-time events. (christmas)
Faster Growth: Merchants with a higher CMGR are expanding their revenue base more quickly. This growth can be driven by factors such as increasing customer acquisition, higher average transaction values, or expanding market reach.
Profit Potential: For the BNPL firm, merchants with a higher CMGR present a better opportunity for long-term profitability since their revenue base is expanding faster. This means the firm could earn more from take rates over time.

In [21]:
sorted_merged_df = sorted_merged_df.drop(columns=['week_count'])
sorted_merged_df

,merchant_abn,cwgr
1,20890410130,0.082710
2,45888399278,0.074185
3,50885695263,0.070904
4,73546158413,0.070117
5,48549026640,0.067869
...,...,...
4417,15293888435,-0.073221
4418,13467303030,-0.076551
4419,86201937910,-0.076596
4420,34205660775,-0.107480


In [22]:
# Export the sorted merged DataFrame to a CSV file
sorted_merged_df.to_csv('../data/curated/cwgr.csv', index=False)